In [11]:
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime

import ast
from collections import Counter

%load_ext autoreload
%autoreload 5
import sys
sys.path.insert(0,"/Users/jdklee/Documents/AbbVie/gcp")
# from gcpReader import *
# from xgbCustom import *
from merger import *
from reader import *
from encoder import *
"""
TODO:
0. deal with labels.. one hot encode to form a list? not sure
1. knn impute missing 
"""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'\nTODO:\n0. deal with labels.. one hot encode to form a list? not sure\n1. knn impute missing \n'

In [12]:
"/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DRUG19Q1.txt".replace("/jdklee/Downloads/",'/lee.j.71/Documents/AbbVie Project/Data/')

'/Users/lee.j.71/Documents/AbbVie Project/Data/faers_ascii_2019Q1/ascii/DRUG19Q1.txt'

In [13]:
#/lee.j.71/Documents/AbbVie Project/Data/

drugs=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DRUG19Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/DRUG19Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/DRUG19Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/DRUG20Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/DRUG20Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/DRUG20Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/DRUG20Q4.txt",
       
      "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/DRUG18Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/DRUG18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/DRUG18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/DRUG18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/DRUG17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/DRUG17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/DRUG17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/DRUG17Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/DRUG16Q1.txt",
    #   "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/DRUG16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/DRUG16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/DRUG16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/DRUG15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/DRUG15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/DRUG15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/DRUG15Q4.txt",
      
            "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/DRUG14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/DRUG14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/DRUG14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/DRUG14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/DRUG13Q1.txt",
       #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/DRUG13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ascii/DRUG13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/DRUG13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/DRUG12Q4.txt"]

demographics=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DEMO19Q1.txt",
         "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/DEMO19Q2.txt",
         "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/DEMO19Q4.txt",
              
         "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/DEMO20Q1.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/DEMO20Q2.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/DEMO20Q3.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/DEMO20Q4.txt",
              
                   
        "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/DEMO18Q1_new.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/DEMO18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/DEMO18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/DEMO18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/DEMO17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/DEMO17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/DEMO17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/DEMO17Q4.txt", 
              
        "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/DEMO16Q1.txt",
#        "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/DEMO16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/DEMO16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/DEMO16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/DEMO15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/DEMO15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/DEMO15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/DEMO15Q4.txt",
             
       "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/DEMO14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/DEMO14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/DEMO14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/DEMO14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/DEMO13Q1.txt",
       #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/DEMO13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ascii/DEMO13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/DEMO13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/DEMO12Q4.txt"]

reactions=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/REAC19Q1.txt",
      "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/REAC19Q2.txt",
      "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/REAC19Q4.txt",
           
      "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/REAC20Q1.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/REAC20Q2.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/REAC20Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/REAC20Q4.txt",
          
        "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/REAC18Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/REAC18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/REAC18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/REAC18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/REAC17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/REAC17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/REAC17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/REAC17Q4.txt",
          
          "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/REAC16Q1.txt",
#        "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/REAC16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/REAC16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/REAC16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/REAC15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/REAC15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/REAC15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/REAC15Q4.txt",
           "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/REAC14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/REAC14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/REAC14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/REAC14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/REAC13Q1.txt",
      #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/REAC13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ascii/REAC13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/REAC13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/REAC12Q4.txt"]   

# drugs=[i.replace("/jdklee/Downloads/",'/lee.j.71/Documents/AbbVie Project/Data/') for i in drugs]
# demographics=[i.replace("/jdklee/Downloads/",'/lee.j.71/Documents/AbbVie Project/Data/') for i in demographics]
# reactions=[i.replace("/jdklee/Downloads/",'/lee.j.71/Documents/AbbVie Project/Data/') for i in reactions]
# drugs=[i.replace("/jdklee/Downloads/",'/gcp/') for i in drugs]
# demographics=[i.replace("/jdklee/Downloads/",'/gcp/') for i in demographics]
# reactions=[i.replace("/jdklee/Downloads/",'/gcp/') for i in reactions]
# [pd.read_csv(i,sep='$', lineterminator='\r') for i in drugs]
# [pd.read_csv(i,sep='$', lineterminator='\r') for i in demographics]
# [pd.read_csv(i,sep='$', lineterminator='\r') for i in reactions]



In [10]:
#GCP VERSION

import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime
from reader import *
from merger import *
import ast
from collections import Counter
from encoder import *


drugs=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DRUG19Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/DRUG19Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/DRUG19Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/DRUG20Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/DRUG20Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/DRUG20Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/DRUG20Q4.txt",
       
      "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/DRUG18Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/DRUG18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/DRUG18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/DRUG18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/DRUG17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/DRUG17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/DRUG17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/DRUG17Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/DRUG16Q1.txt",
    #   "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/DRUG16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/DRUG16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/DRUG16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/DRUG15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/DRUG15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/DRUG15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/DRUG15Q4.txt",
      
            "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/DRUG14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/DRUG14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/DRUG14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/DRUG14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/DRUG13Q1.txt",
       #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/DRUG13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ASCII/DRUG13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/DRUG13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/drug12q4.txt"]

demographics=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DEMO19Q1.txt",
         "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/DEMO19Q2.txt",
         "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/DEMO19Q4.txt",
              
         "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/DEMO20Q1.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/DEMO20Q2.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/DEMO20Q3.txt",
         "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/DEMO20Q4.txt",
              
                   
        "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/DEMO18Q1_new.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/DEMO18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/DEMO18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/DEMO18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/DEMO17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/DEMO17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/DEMO17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/DEMO17Q4.txt", 
              
        "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/DEMO16Q1.txt",
#        "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/DEMO16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/DEMO16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/DEMO16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/DEMO15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/DEMO15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/DEMO15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/DEMO15Q4.txt",
             
       "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/DEMO14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/DEMO14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/DEMO14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/DEMO14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/DEMO13Q1.txt",
       #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/DEMO13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ASCII/DEMO13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/DEMO13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/demo12q4.txt"]

reactions=["/Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/REAC19Q1.txt",
      "/Users/jdklee/Downloads/faers_ascii_2019Q2/ascii/REAC19Q2.txt",
      "/Users/jdklee/Downloads/faers_ascii_2019Q4/ascii/REAC19Q4.txt",
           
      "/Users/jdklee/Downloads/faers_ascii_2020Q1/ASCII/REAC20Q1.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q2/ASCII/REAC20Q2.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q3/ASCII/REAC20Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2020Q4/ASCII/REAC20Q4.txt",
          
        "/Users/jdklee/Downloads/faers_ascii_2018q1/ascii/REAC18Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q2/ascii/REAC18Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q3/ascii/REAC18Q3.txt",
       "/Users/jdklee/Downloads/faers_ascii_2018q4/ascii/REAC18Q4.txt",
       
       "/Users/jdklee/Downloads/faers_ascii_2017q1/ascii/REAC17Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q2/ascii/REAC17Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2017q3/ascii/REAC17Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2017q4/ascii/REAC17Q4.txt",
          
          "/Users/jdklee/Downloads/faers_ascii_2016q1/ascii/REAC16Q1.txt",
#        "/Users/jdklee/Downloads/faers_ascii_2016q2/ascii/REAC16Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2016q3/ascii/REAC16Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2016q4/ascii/REAC16Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2015q1/ascii/REAC15Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q2/ascii/REAC15Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2015q3/ascii/REAC15Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2015q4/ascii/REAC15Q4.txt",
           "/Users/jdklee/Downloads/faers_ascii_2014q1/ascii/REAC14Q1.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q2/ascii/REAC14Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2014q3/ascii/REAC14Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2014q4/ascii/REAC14Q4.txt",
      
      "/Users/jdklee/Downloads/faers_ascii_2013q1/ascii/REAC13Q1.txt",
      #"/Users/jdklee/Downloads/faers_ascii_2013q2/ascii/REAC13Q2.txt",
       "/Users/jdklee/Downloads/faers_ascii_2013q3/ASCII/REAC13Q3.txt",
      "/Users/jdklee/Downloads/faers_ascii_2013q4/ascii/REAC13Q4.txt",
      
       "/Users/jdklee/Downloads/faers_ascii_2012q4/ascii/reac12q4.txt"]   

drugs=[i.replace("/Users/jdklee/Downloads/",'/home/jdklee/gcp/') for i in drugs]
demographics=[i.replace("/Users/jdklee/Downloads/",'/home/jdklee/gcp/') for i in demographics]
reactions=[i.replace("/Users/jdklee/Downloads/",'/home/jdklee/gcp/') for i in reactions]




rough_df=pd.read_csv("/home/jdklee/gcp/aggregate_full.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]
r=[]
[r.extend(ast.literal_eval(ai)) for ai in rough_df.pt]
r=dict(Counter(r))
reactionDict={k: v for k, v in sorted(r.items(), key=lambda item: item[1], reverse=True) if v>1000}

ingredients=[]
[ingredients.extend(ast.literal_eval(ai)) for ai in rough_df.prod_ai]
ingredients=dict(Counter(ingredients))
aiDict={k: v for k, v in sorted(ingredients.items(), key=lambda item: item[1], reverse=True) if v>10}

labels=[i for i in reactionDict.keys()]
for r in labels:
    a=dataReader(chunkSize=50000,  
                 drugs=drugs, 
                 reactions=reactions, 
                 demographics=demographics, 
                 mode=r)        
    a.multiprocessor()
    df=pd.read_csv("aggregate_{}.csv".format(r))
    df=df[df.pt!="pt"].drop("caseid_y", axis=1).reset_index(drop=True)
    oneHotEncoder(df=df, aiThreshold=100, 
                  labelThreshold=100, savePath="{}OneHotEncoded.csv".format(r), 
                  mode="reaction",
                 hotcode=list(aiDict.keys())).run()


pd.concat([pd.read_csv("/home/jdklee/gcp/{}OneHotEncoded.csv".format(r)) for r in labels]).fillna(0).to_csv("complete_onehot.csv",
                                                                                 index=False)

FileNotFoundError: [Errno 2] File b'/home/jdklee/gcp/aggregate_full.csv' does not exist: b'/home/jdklee/gcp/aggregate_full.csv'

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import ast
from collections import Counter
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime
import ast
from collections import Counter
from numpy import loadtxt
from xgboost import XGBClassifier
 
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import sys
sys.path.insert(0,"/home/jdklee/gcp")
from gcpReader import *
from xgbCustom import *

aiDictPath="gs://abbvie_data_one/aggregate_full.csv"
rough_df = pd.read_csv(aiDictPath).dropna().reset_index(drop=True)
rough_df = rough_df[rough_df.primaryid != "primaryid"]
r = []
[r.extend(ast.literal_eval(ai)) for ai in rough_df.pt]
r = dict(Counter(r))
reactionDict = {k: v for k, v in sorted(r.items(), key=lambda item: item[1], reverse=True) if v > 1000}

    
    
for target in reactionDict.keys():
    
    df=onehotReader(aiDictPath="gs://abbvie_data_one/aggregate_full.csv", binary=True, target=target).read()
    
    a=xgbCustom(df=df,
              featureThreshold=1000,
              featureReductionThreshold=500,
              reductionMercyThreshold=5,
              binary=True,
              target=target)
    a.optimize_run()

In [7]:
import ast
rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_full.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]
r=[]
[r.extend(ast.literal_eval(ai)) for ai in rough_df.pt]
r=dict(Counter(r))
reactionDict={k: v for k, v in sorted(r.items(), key=lambda item: item[1], reverse=True) if v>1000}

ingredients=[]
[ingredients.extend(ast.literal_eval(ai)) for ai in rough_df.prod_ai]
ingredients=dict(Counter(ingredients))
aiDict={k: v for k, v in sorted(ingredients.items(), key=lambda item: item[1], reverse=True) if v>10}


len(aiDict),len(reactionDict)

/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(4407, 758)

In [ ]:
#pts=["Neutropenia","Nausea","Diarrhoea","Dizziness"]
#labels=[i for i in reactionDict.keys() if i not in pts]

labels=[i for i in reactionDict.keys()]


for r in labels:
    a=dataReader(chunkSize=50000,  
                 drugs=drugs, 
                 reactions=reactions, 
                 demographics=demographics, 
                 mode=r)        
    a.multiprocessor()
    df=pd.read_csv("aggregate_{}.csv".format(r))
    df=df[df.pt!="pt"].drop("caseid_y", axis=1).reset_index(drop=True)
    oneHotEncoder(df=df, aiThreshold=100, 
                  labelThreshold=100, savePath="{}OneHotEncoded.csv".format(r), 
                  mode=r,
                 hotcode=list(aiDict.keys())).run()




time taken to process Nausea: -335.37928223609924
No drop encoder
encoding active ingredients


In [4]:
        
a=dataReader(chunkSize=50000, mode="pair", 
             drugs=drugs, reactions=reactions, demographics=demographics)
        
a.multiprocessor()

# rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_pair.csv").dropna().reset_index(drop=True)
# rough_df=rough_df[rough_df.primaryid!="primaryid"]
# meanEncoder(df=rough_df, aiThreshold=1000, labelThreshold=1000, savePath="completeDatasetPair.csv", mode="pair")

        

/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Mean Encoder setup
init starting


/Users/jdklee/Documents/AbbVie/merger.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo["primaryid"]=demo.primaryid.apply(lambda x: int(float(x)))
/Users/jdklee/Documents/AbbVie/merger.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo["caseid"]=demo.caseid.apply(lambda x: int(float(x)))
/Users/jdklee/Documents/AbbVie/merger.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

dataSetup init done
pair merge in prog
pairwaise merger starting
Time taken = 3874.6061940193176 seconds
merge drug pair done
demo merged with drug
reaction merged
drug merge complete, saving to csv
save complete for 0
Mean Encoder setup
init starting
dataSetup init done
pair merge in prog
pairwaise merger starting
Time taken = 3964.4520490169525 seconds
merge drug pair done
demo merged with drug
reaction merged
drug merge complete, saving to csv
save complete for 1
Mean Encoder setup
init starting
dataSetup init done
pair merge in prog
pairwaise merger starting
Time taken = 3655.7502081394196 seconds
merge drug pair done
demo merged with drug
reaction merged
drug merge complete, saving to csv
save complete for 2
Mean Encoder setup
init starting
dataSetup init done
pair merge in prog
pairwaise merger starting
Time taken = 0.23994803428649902 seconds
merge drug pair done
demo merged with drug
reaction merged
drug merge complete, saving to csv
save complete for 3
Mean Encoder setup
init 

/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0,13) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

n:26
 /Users/jdklee/Downloads/faers_ascii_2013q1/ascii/DRUG13Q1.txt 
 Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'val_vbm',
       'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit', 'dechal',
       'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt', 'dose_unit',
       'dose_form', 'dose_freq'],
      dtype='object')


/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Mean Encoder setup
failed for 26
n:27
 /Users/jdklee/Downloads/faers_ascii_2013q3/ascii/DRUG13Q3.txt 
 Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'val_vbm',
       'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit', 'dechal',
       'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt', 'dose_unit',
       'dose_form', 'dose_freq'],
      dtype='object')
Mean Encoder setup
failed for 27


/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


n:28
 /Users/jdklee/Downloads/faers_ascii_2013q4/ascii/DRUG13Q4.txt 
 Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'val_vbm',
       'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit', 'dechal',
       'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt', 'dose_unit',
       'dose_form', 'dose_freq'],
      dtype='object')
Mean Encoder setup
failed for 28
n:29
 /Users/jdklee/Downloads/faers_ascii_2012q4/ascii/DRUG12Q4.txt 
 Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'val_vbm',
       'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit', 'dechal',
       'rechal', 'lot_nbr', 'exp_dt', 'nda_num', 'dose_amt', 'dose_unit',
       'dose_form', 'dose_freq'],
      dtype='object')
Mean Encoder setup
failed for 29


In [2]:
rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_pair.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]
meanEncoder(df=rough_df, aiThreshold=1000, labelThreshold=1000, savePath="completeDatasetPair.csv", mode="pair")



/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NameError: name 'meanEncoder' is not defined

In [ ]:
a=dataReader(chunkSize=50000, mode="single", 
             drugs=drugs, reactions=reactions, demographics=demographics)
        
a.multiprocessor()

/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0,21) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


n:0
 /Users/jdklee/Downloads/faers_ascii_2019Q1/ascii/DEMO19Q1.txt 
 Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt',
       'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr',
       'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod',
       'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country'],
      dtype='object')
Mean Encoder setup
failed for 0
Mean Encoder setup
init starting


In [1]:
        
a=dataReader(chunkSize=50000, mode="single", 
             drugs=drugs[:2], reactions=reactions[:2], demographics=demographics[:2])
        
a.multiprocessor()
rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_single.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]
meanEncoder(df=rough_df, aiThreshold=1000, labelThreshold=1000, savePath="completeDatasetPair.csv", mode="single")

        

NameError: name 'dataReader' is not defined

In [2]:

rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_full.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]
rough_df

/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,primaryid,caseid,age,sex,wt,prod_ai,pt
3,1000808590,10008085,4.0,2,50.34,"['OCTREOTIDE ACETATE', 'DILTIAZEM HYDROCHLORID...","['Abdominal adhesions', 'Anaemia', 'Arrhythmia..."
4,100157133,10015713,4.0,2,71.2,"['ATORVASTATIN CALCIUM', 'LEVOTHYROXINE SODIUM']","['Arthralgia', 'Hypokinesia', 'Muscle disorder..."
5,100172312,10017231,3.0,2,80.0,['ALGLUCOSIDASE ALFA'],"['Accident', 'Back injury', 'Fall', 'Joint inj..."
6,100244892,10024489,3.0,2,94.33,"['PREGABALIN', 'OXYCODONE']","['Arthritis', 'Balance disorder', 'Foot deform..."
7,100292857,10029285,4.0,2,80.7,['FINGOLIMOD HYDROCHLORIDE'],"['Blood potassium decreased', 'Brain natriuret..."
...,...,...,...,...,...,...,...
1098821,99966732,9996673,3,2,82,['ATORVASTATIN CALCIUM'],"['Pancreatitis', 'Pulmonary thrombosis', 'Type..."
1098822,99975062,9997506,3,2,102,['ATORVASTATIN CALCIUM'],"['Neoplasm malignant', 'Type 2 diabetes mellit..."
1098823,99976382,9997638,3,2,88,['ATORVASTATIN CALCIUM'],['Type 2 diabetes mellitus']
1098824,99996483,9999648,3,2,62,['DASATINIB'],"['Abortion induced', 'Maternal exposure during..."


In [3]:

oneHotEncoder(df=rough_df, aiThreshold=10, labelThreshold=10, savePath="completeDataset.csv")

        

setting up columns


KeyboardInterrupt: 

In [ ]:
### For iPython
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime
from reader import dataReader
from merger import dataSetup, AImerger
import ast
from collections import Counter
from encoder import *

rough_df=pd.read_csv("/Users/jdklee/Documents/AbbVie/aggregate_full.csv").dropna().reset_index(drop=True)
rough_df=rough_df[rough_df.primaryid!="primaryid"]

oneHotEncoder(df=rough_df.iloc[:10], aiThreshold=0, labelThreshold=0, savePath="completeDataset.csv")

        
        

In [2]:
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime
from reader import dataReader
from merger import dataSetup, AImerger
import ast
from collections import Counter
from encoder import *        
a=dataReader(chunkSize=50000,  
             drugs=drugs, reactions=reactions, demographics=demographics,mode="Neutropenia")
        
a.multiprocessor()

In [1]:
import pandas as pd
import numpy as np
import time
import multiprocessing
from datetime import datetime
from reader import dataReader
from merger import dataSetup, AImerger
import ast
from collections import Counter
from encoder import *  
neut=pd.read_csv("aggregate_Neutropenia.csv")
neut=neut[neut.pt!="pt"].drop("caseid_y", axis=1).reset_index(drop=True)
neut

,primaryid,caseid_x,pt,age,sex,wt,prod_ai
0,1020959017,10209590,Neutropenia,3.0,1,49.0,"['TOCILIZUMAB', 'ACETAMINOPHEN\\CODEINE PHOSPH..."
1,1052511412,10525114,Neutropenia,3.0,1,81.0,"['DOCETAXEL', 'TRASTUZUMAB', 'PERTUZUMAB', 'DE..."
2,1110453913,11104539,Neutropenia,3.0,1,70.0,"['FINGOLIMOD HYDROCHLORIDE', 'ZOLPIDEM TARTRAT..."
3,118035606,11803560,Neutropenia,4.0,0,119.3,['LENALIDOMIDE']
4,118745632,11874563,Neutropenia,3.0,0,109.7,"['SECUKINUMAB', 'METHOTREXATE']"
...,...,...,...,...,...,...,...
10117,97964893,9796489,Neutropenia,3.0,0,53.5,"['MARAVIROC', 'RALTEGRAVIR', 'LOPINAVIR\\RITON..."
10118,97977513,9797751,Neutropenia,3.0,0,51.0,"['PENTOSAN POLYSULFATE SODIUM', 'EMTRICITABINE..."
10119,99170715,9917071,Neutropenia,4.0,0,81.0,"['BOCEPREVIR', 'PEGINTERFERON ALFA-2B', 'RIBAV..."
10120,99808226,9980822,Neutropenia,3.0,0,82.0,"['BEVACIZUMAB', 'PEMETREXED', 'CARBOPLATIN', '..."


In [4]:
reactions=["Neutropenia","Nausea","Diarrhoea","Dizziness"]
for r in reactions:
    a=dataReader(chunkSize=50000,  
                 drugs=drugs, reactions=reactions, demographics=demographics,mode=r)        
    a.multiprocessor()
    df=pd.read_csv("aggregate_{}.csv".format(r))
    df=df[df.pt!="pt"].drop("caseid_y", axis=1).reset_index(drop=True)
    oneHotEncoder(df=df, aiThreshold=5, labelThreshold=0, savePath="{}OneHotEncoded.csv".format(r), mode="reaction")



/Users/jdklee/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: [Errno 2] File b'Neutropenia' does not exist: b'Neutropenia'

In [5]:
neutFinal=pd.read_csv("NeutropeniaOneHotEncoded.csv")
neutFinal=neutFinal[neutFinal.pt!="pt"]
neutFinal

,.ALPHA.-LIPOIC ACID,.ALPHA.-TOCOPHEROL,ABACAVIR SULFATE\LAMIVUDINE,ABATACEPT,ABEMACICLIB,ABIRATERONE ACETATE,ACARBOSE,ACECLOFENAC,ACETAMINOPHEN,ACETAMINOPHEN\ASPIRIN\CAFFEINE,...,ZIV-AFLIBERCEPT,ZOLEDRONIC ACID,ZOLMITRIPTAN,ZOLPIDEM TARTRATE,ZOLPIDEM\ZOLPIDEM TARTRATE,ZOPICLONE,age,pt,sex,wt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,Neutropenia,1,49.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,Neutropenia,1,81.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,3.0,Neutropenia,1,70.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Neutropenia,1,98.6
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Neutropenia,0,119.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,Neutropenia,1,96
10127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,Neutropenia,0,77.1
10128,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,Neutropenia,0,66.2
10129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,Neutropenia,0,42.6


In [6]:
neutFinal.ABATACEPT.value_counts()

0.0    9608
0.0     503
1.0      11
Name: ABATACEPT, dtype: int64

In [9]:
a=pd.read_csv("completeDataset.csv")
a=a[a.pt!="pt"]
a

,"(1-743)-(1638-2332)-BLOOD-COAGULATION FACTOR VIII (SYNTHETIC HUMAN) FUSION PROTEIN WITH IMMUNOGLOBULIN G1 (SYNTHETIC HUMAN FC DOMAIN FRAGMENT), (1444-6'),(1447-9')-BIS(DISULFIDE) WITH IMMUNOGLOBULIN G1 (SYNTHETIC HUMAN FC DOMAIN FRAGMENT)",.ALPHA.-ACETYLDIGOXIN,.ALPHA.-GLUCOSE-1-PHOSPHATE,.ALPHA.-LIPOIC ACID,.ALPHA.-TOCOPHEROL,.ALPHA.-TOCOPHEROL ACETATE,".ALPHA.-TOCOPHEROL ACETATE, D-",".ALPHA.-TOCOPHEROL ACETATE, DL-\.BETA.-CAROTENE\ASCORBIC ACID\CALCIUM CARBONATE\CHOLECALCIFEROL\CUPRIC OXIDE\CYANOCOBALAMIN\FERROUS FUMARATE\FOLIC ACID\NIACINAMIDE\PYRIDOXINE HYDROCHLORIDE\RIBOFLAVIN\THIAMINE MONONITRATE\VITAMIN A ACETATE\ZINC OXIDE",".ALPHA.-TOCOPHEROL ACETATE, DL-\ASCORBIC ACID\CYANOCOBALAMIN\FLUORIDE ION\FOLIC ACID\NIACIN\PYRIDOXINE\RIBOFLAVIN\THIAMINE\VITAMIN A\VITAMIN D",".ALPHA.-TOCOPHEROL, D-",...,ZOPICLONE,ZOTEPINE,ZUCLOPENTHIXOL,ZUCLOPENTHIXOL ACETATE,ZUCLOPENTHIXOL DECANOATE,ZUCLOPENTHIXOL HYDROCHLORIDE,age,pt,sex,wt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Abdominal adhesions,2,50.34
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Anaemia,2,50.34
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Arrhythmia,2,50.34
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Arthralgia,2,50.34
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Atrial fibrillation,2,50.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,Vertigo,1,83.9
8010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,Head injury,2,71
8011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,Blood triglycerides increased,1,95.24
8012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,Weight decreased,2,55.78


In [8]:
{k: v for k, v in sorted(r.items(), key=lambda item: item[1], reverse=True)}

{'Nausea': 66183,
 'Drug ineffective': 61966,
 'Fatigue': 57838,
 'Diarrhoea': 55564,
 'Headache': 54117,
 'Dyspnoea': 54114,
 'Pain': 45234,
 'Dizziness': 44757,
 'Off label use': 43787,
 'Vomiting': 42611,
 'Malaise': 39107,
 'Pyrexia': 34886,
 'Asthenia': 34093,
 'Arthralgia': 31767,
 'Weight decreased': 30474,
 'Pneumonia': 29431,
 'Pain in extremity': 28676,
 'Fall': 27722,
 'Rash': 26351,
 'Product use issue': 25526,
 'Anxiety': 25314,
 'Insomnia': 25282,
 'Pruritus': 24776,
 'Cough': 24720,
 'Abdominal pain': 23995,
 'Decreased appetite': 23884,
 'Alopecia': 22642,
 'Acute kidney injury': 22476,
 'Weight increased': 21665,
 'Feeling abnormal': 21428,
 'Back pain': 21074,
 'Condition aggravated': 20807,
 'Anaemia': 20461,
 'Depression': 18894,
 'Hypotension': 18511,
 'Gait disturbance': 18093,
 'Abdominal pain upper': 18012,
 'Chest pain': 17763,
 'Death': 17479,
 'Peripheral swelling': 17303,
 'Hypertension': 17101,
 'Somnolence': 16670,
 'Constipation': 16496,
 'Myalgia': 16124

In [ ]:
mode=False
if not mode:
    print(0)